<a href="https://colab.research.google.com/github/Dipak22/DeepLearning/blob/master/drug_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2025-02-23 14:29:50--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [              <=>   ]  41.00M  10.5MB/s    in 4.9s    

2025-02-23 14:29:56 (8.42 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

data_files = {'train':'drugsComTrain_raw.tsv', 'test':'drugsComTest_raw.tsv'}

drug_dataset = load_dataset('csv', data_files=data_files, delimiter='\t')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
drug_sample = drug_dataset['train'].shuffle(seed=42).select(range(1000))
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [ ]:
for split in drug_dataset.keys():
  assert len(drug_dataset[split]) == len(drug_dataset[split].unique('Unnamed: 0'))

In [ ]:
drug_dataset = drug_dataset.rename_column(
    original_column_name='Unnamed: 0', new_column_name='patient_id'
)
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [ ]:
for split in drug_dataset.keys():
  print(f"{split} : drugs: {len(drug_dataset[split].unique('drugName'))} , condition : {len(drug_dataset[split].unique('condition'))}")


train : drugs: 3436 , condition : 885
test : drugs: 2637 , condition : 709


In [ ]:
def lowercase_condition(example):
  return {'condition': example['condition'].lower()}



In [ ]:
def filter_nones(x):
  return x['condition'] is not None

In [ ]:
drug_dataset = drug_dataset.filter(filter_nones)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [ ]:
drug_dataset = drug_dataset.map(lowercase_condition)
drug_dataset['train']['condition'][:3]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

['left ventricular dysfunction', 'adhd', 'birth control']

In [ ]:
def compute_review_length(example):
  return {'review_length': len(example['review'].split())}

In [ ]:
drug_dataset = drug_dataset.map(compute_review_length)
drug_dataset['train'][0]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [ ]:
drug_dataset['train'].sort('review_length')[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [ ]:
drug_dataset = drug_dataset.filter(lambda x: x['review_length']>30)
drug_dataset.num_rows

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}

In [ ]:
import html
text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [ ]:
drug_dataset = drug_dataset.map(lambda x: {'review': html.unescape(x['review'])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize_function(examples):
  return tokenizer(examples['review'], truncation= True)

In [ ]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 25.8 s, sys: 207 ms, total: 26 s
Wall time: 19.8 s


In [ ]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [ ]:
# Get unique conditions and assign each a unique integer
unique_conditions = tokenized_dataset["train"].unique("condition")
unique_conditions +=(tokenized_dataset["test"].unique("condition"))
condition_to_int = {condition: idx for idx, condition in enumerate(unique_conditions)}

print(condition_to_int)  # Example output: {'Diabetes': 0, 'Hypertension': 1, 'Asthma': 2}


{'adhd': 848, 'birth control': 822, 'opiate dependence': 881, 'benign prostatic hyperplasia': 845, 'bipolar disorde': 833, 'epilepsy': 952, 'migraine prevention': 824, 'depression': 819, "crohn's disease": 909, 'cough': 876, 'obesity': 871, 'urinary tract infection': 911, 'ibromyalgia': 864, 'chronic myelogenous leukemia': 899, 'hiv infection': 903, 'insomnia': 828, 'rheumatoid arthritis': 846, 'vaginal yeast infection': 827, 'chlamydia infection': 971, 'hirsutism': 1036, 'panic disorde': 869, 'migraine': 938, 'pain': 839, 'constipation': 870, 'bowel preparation': 836, 'psychosis': 1110, 'hepatitis c': 970, 'overactive bladde': 956, 'diabetes, type 2': 838, 'schizophrenia': 853, 'dysuria': 1086, 'anxiety': 851, 'acne': 831, 'emale infertility': 900, 'constipation, drug induced': 837, 'erectile dysfunction': 877, 'trigeminal neuralgia': 1073, 'chronic pain': 891, 'atrophic vaginitis': 1063, 'skin and structure infection': 1170, 'emergency contraception': 842, 'tinnitus': 1099, 'major de

In [ ]:
def convert_condition_to_label(example):
    example["labels"] = condition_to_int[example["condition"]]  # Convert to int
    return example

# Apply transformation to all splits
tokenized_dataset = tokenized_dataset.map(convert_condition_to_label)


Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset["train"][0]['labels'])

848


In [ ]:
drug_dataset_clean = tokenized_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = tokenized_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 46108
    })
})

In [ ]:
drug_dataset_clean = drug_dataset_clean.remove_columns(["patient_id", "drugName", "review","date","rating","usefulCount","review_length",'condition'])
drug_dataset_clean.set_format("torch")
drug_dataset_clean["train"].column_names

ValueError: Column name ['rating', 'review_length', 'patient_id', 'date', 'condition', 'drugName', 'review', 'usefulCount'] not in the dataset. Current columns in the dataset: ['input_ids', 'token_type_ids', 'attention_mask', 'labels']

In [ ]:
drug_dataset_clean["train"][0:3]['labels']

tensor([897, 822, 822])

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    drug_dataset_clean["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    drug_dataset_clean["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 46108
    })
})

In [ ]:
for i in range(5):
  for k,v in drug_dataset_clean['train'][i].items():
    if k == 'labels':
      print(v)

tensor(897)
tensor(822)
tensor(822)
tensor(851)
tensor(927)


In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 203]),
 'token_type_ids': torch.Size([8, 203]),
 'attention_mask': torch.Size([8, 203]),
 'labels': torch.Size([8])}

In [ ]:
len(unique_conditions)

1471

In [ ]:
from transformers import AutoModelForSequenceClassification
checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1471)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(7.3706, grad_fn=<NllLossBackward0>) torch.Size([8, 1471])


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

41556


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [1]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

NameError: name 'num_training_steps' is not defined